# New title launch support checklist

Author: @haewon.yum
- This is automated health check for a given bundle, based on new launch check list

**Checks:**

**[Bundle-level Check]**<br>
(1) PA Enabled Postback Reception<br>
(3) Revenue Postback Reception<br>
<br>
**[Campaign-level Check]**<br>
(2) VT Install Reception<br>
(4) Key Creative Format Impressions<br>
(5) (Domestic KOR) Kakao Bizboard (1029x258) Creative<br>
(6) High Bid Filter Rate<br>
(7) CT Install Leakage (1h window)

In [69]:
# #@title colab authentication

# from google.colab import auth
# auth.authenticate_user()


In [70]:
#@title Environment Setup

from google.cloud import bigquery
import pandas as pd
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', 80)

client = bigquery.Client(project='moloco-ods')

def run_query(query, label=''):
    """Run a BQ query and return DataFrame. Print row count."""
    try:
        df = client.query(query).result().to_dataframe()
        status = f'✅ {label}: {len(df)} rows' if len(df) > 0 else f'⚠️ {label}: 0 rows — check needed'
        print(status)
        return df
    except Exception as e:
        print(f'❌ {label}: Query failed — {e}')
        return pd.DataFrame()

In [71]:
#@title Parameters

BUNDLE_ID = 'com.fatmerge.global'  #@param {type:"string"}
CAMPAIGN_ID = 'GK2fIdNthVl1rtyn'  #@param {type:"string"}

# ── Resolve all bundle variants from product_digest ──
# Input can be any of: mmp_bundle_id, tracking_bundle, or app_store_bundle.
# We look up all variants so each query uses the correct identifier.

q_resolve = f"""
WITH bundle_resolve AS (
  SELECT
    pd.app_store_bundle,
    pd.app_tracking_bundle,
    pd.tracking_bundle,
    pd.title AS product_title,
    pd.os,
    mmp.mmp_bundle_id
  FROM `focal-elf-631.standard_digest.product_digest` pd
  LEFT JOIN (
    SELECT DISTINCT
      advertiser.mmp_bundle_id,
      product.app_market_bundle
    FROM `moloco-ae-view.athena.fact_dsp_core`
    WHERE date_utc >= DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY)
      AND advertiser.mmp_bundle_id IS NOT NULL
  ) mmp
    ON pd.app_store_bundle = mmp.app_market_bundle
  WHERE NOT pd.is_archived
    AND (
      pd.app_store_bundle = '{BUNDLE_ID}'
      OR pd.app_tracking_bundle = '{BUNDLE_ID}'
      OR pd.tracking_bundle = '{BUNDLE_ID}'
      OR mmp.mmp_bundle_id = '{BUNDLE_ID}'
    )
)
SELECT DISTINCT * FROM bundle_resolve
"""

df_bundle = run_query(q_resolve, 'Bundle Resolution')

if df_bundle.empty:
    print(f'⚠️ Could not resolve bundle "{BUNDLE_ID}". Using as-is for all queries.')
    TRACKING_BUNDLE = BUNDLE_ID
    MMP_BUNDLE_ID = BUNDLE_ID
    APP_STORE_BUNDLE = BUNDLE_ID
else:
    print(df_bundle.to_string(index=False))
    row = df_bundle.iloc[0]
    TRACKING_BUNDLE = row.get('tracking_bundle') or row.get('app_tracking_bundle') or BUNDLE_ID
    MMP_BUNDLE_ID = row.get('mmp_bundle_id') or BUNDLE_ID
    APP_STORE_BUNDLE = row.get('app_store_bundle') or BUNDLE_ID

APP_OS = df_bundle['os'].iloc[0].upper() if not df_bundle.empty and 'os' in df_bundle.columns else 'UNKNOWN'

print(f'\n── Resolved Bundle IDs ──')
print(f'  tracking_bundle  : {TRACKING_BUNDLE}')
print(f'  mmp_bundle_id    : {MMP_BUNDLE_ID}')
print(f'  app_store_bundle : {APP_STORE_BUNDLE}')
print(f'  os               : {APP_OS}')

# Optional campaign filter clause
_campaign_filter = f"AND api.campaign.id = '{CAMPAIGN_ID}'" if CAMPAIGN_ID else ''
_campaign_filter_core = f"AND campaign_id = '{CAMPAIGN_ID}'" if CAMPAIGN_ID else ''

print(f'\n  Campaign: {CAMPAIGN_ID or "(all campaigns)"}')

✅ Bundle Resolution: 1 rows
   app_store_bundle app_tracking_bundle     tracking_bundle                  product_title      os       mmp_bundle_id
com.fatmerge.global com.fatmerge.global com.fatmerge.global Food and Travel Merge Game AOS ANDROID com.fatmerge.global

── Resolved Bundle IDs ──
  tracking_bundle  : com.fatmerge.global
  mmp_bundle_id    : com.fatmerge.global
  app_store_bundle : com.fatmerge.global
  os               : ANDROID

  Campaign: GK2fIdNthVl1rtyn


---
## 1. PA Enabled Postback Reception

In [72]:
#@title 1-A. PA Status (mmp_pb_summary.app_status — iOS)

q_1a = f"""
SELECT
  utc_date,
  mmp,
  tracking_bundle,
  verdict.fp_status AS pa_status,
  verdict.opt_with_ifa,
  verdict.opt_with_mas,
  warning.appsflyer_ap_on,
  warning.appsflyer_aap_enabled,
  warning.appsflyer_vt_pa_enabled,
  attr.att.total AS attr_att_optin,
  attr.no_att.total AS attr_att_optout,
  attr.no_att.privacy AS attr_privacy_count,
  no_attr.att.total AS noattr_att_optin,
  no_attr.no_att.total AS noattr_att_optout,
  ROUND(spend.total, 2) AS spend_usd
FROM `focal-elf-631.mmp_pb_summary.app_status`
WHERE tracking_bundle = '{TRACKING_BUNDLE}'
  AND utc_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 14 DAY)
ORDER BY utc_date DESC
"""

df_1a = run_query(q_1a, '1-A PA Status (iOS)')
df_1a

⚠️ 1-A PA Status (iOS): 0 rows — check needed


,utc_date,mmp,tracking_bundle,pa_status,opt_with_ifa,opt_with_mas,appsflyer_ap_on,appsflyer_aap_enabled,appsflyer_vt_pa_enabled,attr_att_optin,attr_att_optout,attr_privacy_count,noattr_att_optin,noattr_att_optout,spend_usd


In [73]:
#@title 1-B. PA attribution method from cv postback (iOS + Android)

# Uses structured cv.pb.attribution fields (more reliable than regex on postback URL)
q_1b = f"""
SELECT
  DATE(timestamp) AS date,
  cv.mmp,
  cv.pb.attribution.method AS attribution_method,
  cv.pb.attribution.raw_method AS raw_method,
  COUNT(*) AS install_count
FROM `focal-elf-631.prod_stream_view.cv`
WHERE
  DATE(timestamp) BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY) AND DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)
  AND UPPER(cv.event) = 'INSTALL'
  AND api.product.app.tracking_bundle = '{TRACKING_BUNDLE}'
  {_campaign_filter}
GROUP BY 1, 2, 3, 4
ORDER BY 1 DESC, 5 DESC
"""

df_1b = run_query(q_1b, '1-B PA attribution method')
if not df_1b.empty:
    has_pa = df_1b['attribution_method'].str.contains('probabilistic|modeled|fingerprint', case=False, na=False).any()
    print(f'  → PA postbacks detected: {"YES ✅" if has_pa else "NO ⚠️"}')
    print(f'  → Attribution methods seen: {df_1b["attribution_method"].dropna().unique().tolist()}')
df_1b

✅ 1-B PA attribution method: 21 rows
  → PA postbacks detected: YES ✅
  → Attribution methods seen: ['IDENTIFIER', 'REFERRER', 'PROBABILISTIC']


,date,mmp,attribution_method,raw_method,install_count
0,2026-02-26,ADJUST,IDENTIFIER,device_tag,438
1,2026-02-26,ADJUST,REFERRER,external_click_id,215
2,2026-02-26,ADJUST,PROBABILISTIC,probabilistic_matching,1
3,2026-02-25,ADJUST,IDENTIFIER,device_tag,388
4,2026-02-25,ADJUST,REFERRER,external_click_id,217
5,2026-02-25,ADJUST,PROBABILISTIC,probabilistic_matching,1
6,2026-02-24,ADJUST,IDENTIFIER,device_tag,299
7,2026-02-24,ADJUST,REFERRER,external_click_id,214
8,2026-02-24,ADJUST,PROBABILISTIC,probabilistic_matching,1
9,2026-02-23,ADJUST,IDENTIFIER,device_tag,300


---
## 2. VT Install Reception

In [74]:
#@title 2. VT Install check

q_2 = f"""
SELECT
  DATE(timestamp) AS date,
  cv.view_through AS is_view_through,
  cv.pb.attribution.method AS attribution_method,
  cv.pb.attribution.viewthrough AS pb_viewthrough,
  COUNT(*) AS install_count
FROM `focal-elf-631.prod_stream_view.cv`
WHERE
  DATE(timestamp) BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY) AND DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)
  AND UPPER(cv.event) = 'INSTALL'
  AND api.product.app.tracking_bundle = '{TRACKING_BUNDLE}'
  {_campaign_filter}
GROUP BY 1, 2, 3, 4
ORDER BY 1 DESC, 5 DESC
"""

df_2 = run_query(q_2, '2. VT Install')
if not df_2.empty:
    has_vt = df_2['is_view_through'].any() or df_2['pb_viewthrough'].any()
    print(f'  → VT installs detected: {"YES ✅" if has_vt else "NO ⚠️"}')
df_2

✅ 2. VT Install: 21 rows
  → VT installs detected: NO ⚠️


,date,is_view_through,attribution_method,pb_viewthrough,install_count
0,2026-02-26,False,IDENTIFIER,False,438
1,2026-02-26,False,REFERRER,False,215
2,2026-02-26,False,PROBABILISTIC,False,1
3,2026-02-25,False,IDENTIFIER,False,388
4,2026-02-25,False,REFERRER,False,217
5,2026-02-25,False,PROBABILISTIC,False,1
6,2026-02-24,False,IDENTIFIER,False,299
7,2026-02-24,False,REFERRER,False,214
8,2026-02-24,False,PROBABILISTIC,False,1
9,2026-02-23,False,IDENTIFIER,False,300


---
## 3. Revenue Postback Reception

In [75]:
#@title 3. Revenue postback check

q_3 = f"""
SELECT
  DATE(timestamp) AS date,
  cv.event_pb AS event_name,
  COUNT(*) AS event_count,
  COUNTIF(cv.revenue_usd.amount > 0) AS events_with_revenue,
  ROUND(SUM(cv.revenue_usd.amount), 2) AS total_revenue_usd
FROM `focal-elf-631.prod_stream_view.cv`
WHERE
  DATE(timestamp) BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY) AND DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)
  AND api.product.app.tracking_bundle = '{TRACKING_BUNDLE}'
  {_campaign_filter}
  AND cv.revenue_usd.amount IS NOT NULL
GROUP BY 1, 2
ORDER BY 1 DESC, 5 DESC
"""

df_3 = run_query(q_3, '3. Revenue Postback')
if not df_3.empty:
    total_rev = df_3['total_revenue_usd'].sum()
    print(f'  → Total revenue (L7D): ${total_rev:,.2f}')
    print(f'  → Event types: {df_3["event_name"].unique().tolist()}')
df_3

✅ 3. Revenue Postback: 28 rows
  → Total revenue (L7D): $50,035.68
  → Event types: ['Purchase', 'install', 'reattribution', 'session']


,date,event_name,event_count,events_with_revenue,total_revenue_usd
0,2026-02-26,Purchase,951,951,6580.72
1,2026-02-26,install,654,0,0.00
2,2026-02-26,reattribution,8,0,0.00
3,2026-02-26,session,32000,0,0.00
4,2026-02-25,Purchase,862,862,6165.96
5,2026-02-25,install,606,0,0.00
6,2026-02-25,reattribution,7,0,0.00
7,2026-02-25,session,32086,0,0.00
8,2026-02-24,Purchase,715,715,4477.21
9,2026-02-24,install,514,0,0.00


---
## 4. Key Creative Format Impressions

In [76]:
#@title 4-A. Creatives configured (creative_digest)

q_4a = f"""
WITH product AS (
  SELECT product_id, platform
  FROM `focal-elf-631.standard_digest.product_digest`
  WHERE app_store_bundle = '{APP_STORE_BUNDLE}'
    AND NOT is_archived
)
SELECT
  cd.product_id,
  cd.creative_id,
  cd.creative_title,
  cd.creative_type,
  cd.is_archived,
  cd.timestamp AS last_updated
FROM `focal-elf-631.standard_digest.creative_digest` cd
INNER JOIN product p
  ON cd.product_id = p.product_id
  AND cd.platform = p.platform
ORDER BY cd.is_archived, cd.creative_type, cd.timestamp DESC
"""

df_4a = run_query(q_4a, '4-A Creatives Configured')
if not df_4a.empty:
    active = df_4a[df_4a['is_archived'] == False]
    print(f'  → Active creatives: {len(active)}, Archived: {len(df_4a) - len(active)}')
    print(f'  → Active formats: {active["creative_type"].value_counts().to_dict()}')
df_4a.head(20)

✅ 4-A Creatives Configured: 426 rows
  → Active creatives: 407, Archived: 19
  → Active formats: {'RICH_CUSTOM_HTML': 367, 'VIDEO': 40}


,product_id,creative_id,creative_title,creative_type,is_archived,last_updated
0,r1Sbyk6Bh9zpLySP,s5NsXQvz5nSdr7iy,FATPL0372_BJ透视拖动合成果汁杯_moloco.html,RICH_CUSTOM_HTML,False,2026-02-23 02:57:39.936209+00:00
1,r1Sbyk6Bh9zpLySP,d0GoxMNDrD4V8ygV,FATPL096_BJ反向大西瓜贝壳_Moloco.html,RICH_CUSTOM_HTML,False,2026-02-23 02:57:39.936209+00:00
2,r1Sbyk6Bh9zpLySP,vMRmfLL8ZSYxtLlT,FATPL0216_BJ开宝藏地图_Moloco.html,RICH_CUSTOM_HTML,False,2026-02-23 02:57:39.936209+00:00
3,r1Sbyk6Bh9zpLySP,fMIzs1qhFzorPI2w,FATPL147_BJ金币钻石沙滩黄复刻效果版_Moloco.html,RICH_CUSTOM_HTML,False,2026-02-23 02:57:39.936209+00:00
4,r1Sbyk6Bh9zpLySP,SL1ZA6ApCuRxwH7T,FATPL0513_BJ切角蛋糕放大展示2_moloco.html,RICH_CUSTOM_HTML,False,2026-02-23 02:57:39.936209+00:00
5,r1Sbyk6Bh9zpLySP,Vyyl47EVvwZcgokU,FATPL0198_BJ合成链展示订单运动_Moloco.html,RICH_CUSTOM_HTML,False,2026-02-23 02:57:39.936209+00:00
6,r1Sbyk6Bh9zpLySP,GeCWx9PsgS8TjBDi,FATPL0425_BJ果汁杯大西瓜餐厅订单条FAT背景B版_moloco.html,RICH_CUSTOM_HTML,False,2026-02-23 02:57:39.936209+00:00
7,r1Sbyk6Bh9zpLySP,W4sdV6ysxsszA1h3,FATPL0033_GZ祖玛合成两订单加长休闲版_moloco,RICH_CUSTOM_HTML,False,2026-02-23 02:57:39.936209+00:00
8,r1Sbyk6Bh9zpLySP,NGLBihtNlKv2LaWy,FATPL0603_BJ果汁杯大西瓜餐厅订单条多阶棋子叠加摆放K版_moloco,RICH_CUSTOM_HTML,False,2026-02-23 02:57:39.936209+00:00
9,r1Sbyk6Bh9zpLySP,fOkkBDj2RssnWcMy,FATPL009_BJ棋子下落浓郁咖啡主题_Moloco.html,RICH_CUSTOM_HTML,False,2026-02-23 02:57:39.936209+00:00


In [77]:
#@title 4-B. Impressions by creative format (fact_dsp_creative)

q_4b = f"""
SELECT
  date_utc,
  creative.format AS cr_format,
  COUNT(DISTINCT creative.id) AS n_creatives,
  SUM(impressions) AS impressions,
  ROUND(SUM(gross_spend_usd), 2) AS gross_spend_usd,
  SUM(installs) AS installs
FROM `moloco-ae-view.athena.fact_dsp_creative`
WHERE
  date_utc BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY) AND DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)
  AND advertiser.mmp_bundle_id = '{MMP_BUNDLE_ID}'
  {_campaign_filter_core}
GROUP BY 1, 2
ORDER BY 1 DESC, 4 DESC
"""

df_4b = run_query(q_4b, '4-B Impressions by Format')
if not df_4b.empty:
    fmt_summary = df_4b.groupby('cr_format').agg(
        total_impressions=('impressions', 'sum'),
        total_spend=('gross_spend_usd', 'sum')
    ).sort_values('total_spend', ascending=False)
    print(f'  → Formats serving:')
    for fmt, row in fmt_summary.iterrows():
        print(f'     {fmt}: {row["total_impressions"]:,.0f} imps, ${row["total_spend"]:,.2f} spend')
df_4b

✅ 4-B Impressions by Format: 21 rows
  → Formats serving:
     ri: 2,220,395 imps, $122,939.71 spend
     vi: 1,503,058 imps, $51,653.05 spend
     vb: 247,937 imps, $387.98 spend


,date_utc,cr_format,n_creatives,impressions,gross_spend_usd,installs
0,2026-02-26,ri,239,407124,19303.900000000,561
1,2026-02-26,vi,30,202398,6524.990000000,92
2,2026-02-26,vb,4,25400,15.720000000,1
3,2026-02-25,ri,240,374766,19739.400000000,532
4,2026-02-25,vi,30,175766,5419.690000000,72
5,2026-02-25,vb,4,29373,21.180000000,1
6,2026-02-24,ri,240,321478,16002.190000000,400
7,2026-02-24,vi,30,278332,7190.330000000,113
8,2026-02-24,vb,5,35425,34.330000000,0
9,2026-02-23,ri,241,308205,15897.750000000,380


---
## 5. (Domestic KOR) Kakao Bizboard (1029x258)

In [78]:
#@title 5. Kakao Bizboard 1029x258 check

q_5 = f"""
SELECT
  date_utc,
  creative.format AS cr_format,
  creative.size,
  creative.title AS cr_title,
  creative.id AS cr_id,
  SUM(impressions) AS impressions,
  ROUND(SUM(gross_spend_usd), 2) AS gross_spend_usd
FROM `moloco-ae-view.athena.fact_dsp_creative`
WHERE
  date_utc BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 14 DAY) AND DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)
  AND advertiser.mmp_bundle_id = '{MMP_BUNDLE_ID}'
  {_campaign_filter_core}
  AND campaign.country = 'KOR'
  AND LOWER(exchange) LIKE '%kakao%'
  AND creative.size = '1029x258'
GROUP BY 1, 2, 3, 4, 5
ORDER BY 1 DESC, 7 DESC
"""

df_5 = run_query(q_5, '5. Kakao Bizboard 1029x258')
if df_5.empty:
    print('  → No Bizboard (1029x258) impressions on Kakao.')
    print('    Check: creative uploaded? Under review? Wrong dimensions?')
else:
    print(f'  → Bizboard serving: {df_5["impressions"].sum():,.0f} total imps, ${df_5["gross_spend_usd"].sum():,.2f} total spend')
df_5

⚠️ 5. Kakao Bizboard 1029x258: 0 rows — check needed
  → No Bizboard (1029x258) impressions on Kakao.
    Check: creative uploaded? Under review? Wrong dimensions?


,date_utc,cr_format,size,cr_title,cr_id,impressions,gross_spend_usd


---
## 6. High Bid Filter Rate

⚠️ Requires `CAMPAIGN_ID`. Skip if not provided.

In [79]:
#@title 6-A. Bid filter reasons (pricing table, 1/1000 sampled)

if not CAMPAIGN_ID:
    print('⏭️ Skipped — CAMPAIGN_ID not provided')
    df_6a = pd.DataFrame()
else:
    q_6a = f"""
    WITH pricing_data AS (
      SELECT
        DATE(timestamp) AS date,
        cand.candidate_result,
        cand.core.reason AS core_reason,
        COUNT(*) AS cnt,
        # ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (PARTITION BY date), 2) AS pct_of_day
      FROM `focal-elf-631.prod_stream_view.pricing`,
        UNNEST(pricing.candidates) AS cand
      WHERE
        DATE(timestamp) BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY) AND DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)
        AND cand.campaign_id = '{CAMPAIGN_ID}'
      GROUP BY 1, 2, 3
      ORDER BY 1 DESC, 4 DESC
    )
    SELECT *,
      ROUND(100.0 * cnt / SUM(cnt) OVER (PARTITION BY date), 2) AS pct_of_day
    FROM pricing_data
    """

    df_6a = run_query(q_6a, '6-A Bid Filter (pricing)')
    if not df_6a.empty:
        # Passed = CommitBid or InternalAuctionWinner
        PASSED_RESULTS = {'CommitBid', 'InternalAuctionWinner'}
        passed_mask = df_6a['candidate_result'].isin(PASSED_RESULTS)

        # Daily summary: pass vs filter rate
        daily = df_6a.groupby('date').agg(total=('cnt', 'sum'))
        daily['passed'] = df_6a[passed_mask].groupby('date')['cnt'].sum()
        daily['passed'] = daily['passed'].fillna(0).astype(int)
        daily['filtered'] = daily['total'] - daily['passed']
        daily['filter_rate_pct'] = round(100 * daily['filtered'] / daily['total'], 2)
        print('  → Daily filter rate:')
        for dt, row in daily.iterrows():
            print(f'     {dt}:  total={row["total"]:,.0f}  passed={row["passed"]:,.0f}  filtered={row["filtered"]:,.0f}  filter_rate={row["filter_rate_pct"]:.1f}%')

        # Top candidate_result breakdown (filtered only)
        filtered_df = df_6a[~passed_mask]
        if not filtered_df.empty:
            print(f'\n  → Top filter reasons (all days):')
            top = filtered_df.groupby('candidate_result')['cnt'].sum().sort_values(ascending=False)
            grand_total = filtered_df['cnt'].sum()
            for result, cnt in top.head(10).items():
                print(f'     {result}: {cnt:,.0f} ({100*cnt/grand_total:.1f}%)')

            # Chart: daily portion of filtered reasons (after pricing)
            chart_df = filtered_df.groupby(['date', 'candidate_result'])['cnt'].sum().reset_index()
            daily_total = chart_df.groupby('date')['cnt'].transform('sum')
            chart_df['pct'] = (chart_df['cnt'] / daily_total * 100).round(2)
            chart_df['date'] = chart_df['date'].astype(str)
            fig = px.bar(chart_df, x='date', y='pct', color='candidate_result',
                         title='6-A: Daily Filter Reason Breakdown (after pricing)',
                         labels={'pct': '% of Filtered Requests', 'date': '', 'candidate_result': 'Reason'},
                         hover_data=['cnt'])
            fig.update_layout(barmode='stack', yaxis_ticksuffix='%', legend_title_text='candidate_result',
                              height=450, margin=dict(r=200))
            fig.show()
        else:
            print('\n  → All candidates passed (no filters applied)')
    df_6a

✅ 6-A Bid Filter (pricing): 49 rows
  → Daily filter rate:
     2026-02-20:  total=2,765,256  passed=33,805  filtered=2,731,451  filter_rate=98.8%
     2026-02-21:  total=2,851,328  passed=32,756  filtered=2,818,572  filter_rate=98.8%
     2026-02-22:  total=2,916,831  passed=32,501  filtered=2,884,330  filter_rate=98.9%
     2026-02-23:  total=2,715,621  passed=45,137  filtered=2,670,484  filter_rate=98.3%
     2026-02-24:  total=2,665,792  passed=49,842  filtered=2,615,950  filter_rate=98.1%
     2026-02-25:  total=2,537,640  passed=47,102  filtered=2,490,538  filter_rate=98.1%
     2026-02-26:  total=2,581,486  passed=50,613  filtered=2,530,873  filter_rate=98.0%

  → Top filter reasons (all days):
     FilterByBidfloor: 10,905,208 (58.2%)
     InternalAuctionCandidate: 6,372,609 (34.0%)
     FilterByPricingModel: 1,268,001 (6.8%)
     PostPricingCapByCampaignReturnBased: 136,861 (0.7%)
     FilterByCreativePick: 59,519 (0.3%)


In [80]:
#@title 6-B. Bid filter reasons (campaign_trace, detailed)

if not CAMPAIGN_ID:
    print('⏭️ Skipped — CAMPAIGN_ID not provided')
    df_6b = pd.DataFrame()
else:
    q_6b = f"""
    SELECT
      date,
      campaign,
      reason_block,
      reason,
      reason_raw,
      ROUND(SUM(1 / rate) / 1e6, 2) AS estimated_req_millions
    FROM `moloco-data-prod.younghan.campaign_trace_raw_prod`
    WHERE
      date BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 3 DAY) AND DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)
      AND campaign = '{CAMPAIGN_ID}'
      AND reason_block IN ('Get candidate campaigns', 'Evaluate candidate campaigns', 'get candidate ad_groups')
    GROUP BY 1, 2, 3, 4, 5
    ORDER BY 1 DESC, 6 DESC
    """

    df_6b = run_query(q_6b, '6-B Bid Filter (trace)')
    if not df_6b.empty:
        DETAIL_REASONS = {'(campaign) Ctx', '(campaign) Req', '(ad_group) Ctx', '(ad_group) Req'}
        df_6b['reason_label'] = df_6b.apply(
            lambda r: f"{r['reason']}: {r['reason_raw']}" if r['reason'] in DETAIL_REASONS and pd.notna(r['reason_raw']) else r['reason'],
            axis=1)

        from plotly.subplots import make_subplots
        import plotly.graph_objects as go

        BLOCK_ORDER = ['Get candidate campaigns', 'Evaluate candidate campaigns', 'get candidate ad_groups']
        blocks = [b for b in BLOCK_ORDER if b in df_6b['reason_block'].values]
        fig = make_subplots(rows=len(blocks), cols=1, subplot_titles=blocks, vertical_spacing=0.08)

        legend_seen = set()
        for idx, block in enumerate(blocks):
            block_df = df_6b[df_6b['reason_block'] == block]
            chart_df = block_df.groupby(['date', 'reason_label'])['estimated_req_millions'].sum().reset_index()
            daily_total = chart_df.groupby('date')['estimated_req_millions'].transform('sum')
            chart_df['pct'] = (chart_df['estimated_req_millions'] / daily_total * 100).round(2)
            chart_df['date'] = chart_df['date'].astype(str)

            top_reasons = chart_df.groupby('reason_label')['estimated_req_millions'].sum().sort_values(ascending=False).head(10).index
            chart_df = chart_df[chart_df['reason_label'].isin(top_reasons)]

            for reason in top_reasons:
                r_df = chart_df[chart_df['reason_label'] == reason]
                show = reason not in legend_seen
                legend_seen.add(reason)
                fig.add_trace(go.Bar(
                    x=r_df['date'], y=r_df['pct'], name=reason,
                    hovertemplate=f'{reason}<br>%{{y:.1f}}%<br>%{{customdata[0]:.2f}}M req',
                    customdata=r_df[['estimated_req_millions']].values,
                    legendgroup=reason, showlegend=show,
                ), row=idx+1, col=1)

        fig.update_layout(
            barmode='stack', height=400 * len(blocks),
            title_text='6-B: Daily Filter Reason Breakdown by Pipeline Stage',
            legend_title_text='reason', margin=dict(r=250))
        for i in range(len(blocks)):
            fig.update_yaxes(ticksuffix='%', row=i+1, col=1)
        fig.show()
    df_6b

✅ 6-B Bid Filter (trace): 1132 rows


In [81]:
df_6b

,date,campaign,reason_block,reason,reason_raw,estimated_req_millions,reason_label
0,2026-02-26,GK2fIdNthVl1rtyn,Get candidate campaigns,no compatible creatives,no compatible creatives,2766.13,no compatible creatives
1,2026-02-26,GK2fIdNthVl1rtyn,Get candidate campaigns,(campaign) Req,IdTypes,488.44,(campaign) Req: IdTypes
2,2026-02-26,GK2fIdNthVl1rtyn,Get candidate campaigns,category_blocked,category_blocked,164.40,category_blocked
3,2026-02-26,GK2fIdNthVl1rtyn,Get candidate campaigns,blocked advertiser or app,blocked advertiser (adomain),151.30,blocked advertiser or app
4,2026-02-26,GK2fIdNthVl1rtyn,Get candidate campaigns,(campaign) Req,Exchanges,134.10,(campaign) Req: Exchanges
...,...,...,...,...,...,...,...
1127,2026-02-24,GK2fIdNthVl1rtyn,Evaluate candidate campaigns,campaign limiter,"mu: -6.895049, adj_mu: -3.200180, prc_cnt: 1, explore",0.00,campaign limiter
1128,2026-02-24,GK2fIdNthVl1rtyn,get candidate ad_groups,avoided pricing (per cr_format) with limits,"avoided pricing (per cr_format) with upper-limit (600000), lower-limit (7118...",0.00,avoided pricing (per cr_format) with limits
1129,2026-02-24,GK2fIdNthVl1rtyn,Evaluate candidate campaigns,campaign limiter,"mu: -5.708882, adj_mu: -2.077247, prc_cnt: 2, explore",0.00,campaign limiter
1130,2026-02-24,GK2fIdNthVl1rtyn,Evaluate candidate campaigns,campaign limiter,"mu: -7.470036, adj_mu: -4.309295, prc_cnt: 6, explore",0.00,campaign limiter


---
## 7. Install Leakage / Rejected Install Rate

In [82]:
#@title 7-A. CT Install Leakage — unattributed installs within 1h of Moloco click

q_7a = f"""
WITH
  install AS (
    SELECT
      device.ifa,
      MAX(timestamp) AS install_ts
    FROM `focal-elf-631.prod_stream_view.pb`
    WHERE DATE(timestamp) BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 14 DAY) AND CURRENT_DATE()
      AND app.bundle = '{TRACKING_BUNDLE}'
      AND LOWER(event.name) = 'install'
      AND moloco.attributed IS FALSE
      AND `moloco-ods.general_utils.is_userid_truly_available`(device.ifa)
    GROUP BY 1
  ),
  click_matched AS (
    SELECT
      c.req.device.ifa,
      MAX(c.timestamp) AS last_click
    FROM `focal-elf-631.prod_stream_view.click` c
    INNER JOIN install i ON i.ifa = c.req.device.ifa
    WHERE c.timestamp < i.install_ts
      AND DATE(c.timestamp) BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 15 DAY) AND CURRENT_DATE()
      AND c.api.product.app.tracking_bundle = '{TRACKING_BUNDLE}'
      {"AND c.api.campaign.id = '" + CAMPAIGN_ID + "'" if CAMPAIGN_ID else ""}
    GROUP BY 1
  ),
  leaked AS (
    SELECT
      DATE(i.install_ts) AS date,
      i.ifa
    FROM click_matched c
    INNER JOIN install i USING (ifa)
    WHERE i.install_ts > c.last_click
      AND TIMESTAMP_DIFF(i.install_ts, c.last_click, MINUTE) BETWEEN 0 AND 60
  ),
  leaked_daily AS (
    SELECT date, COUNT(DISTINCT ifa) AS leaked_installs
    FROM leaked
    GROUP BY 1
  ),
  moloco_attributed AS (
    SELECT
      date_utc AS date,
      SUM(installs) AS attributed_installs
    FROM `moloco-ae-view.athena.fact_dsp_core`
    WHERE date_utc BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 14 DAY) AND DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)
      AND advertiser.mmp_bundle_id = '{MMP_BUNDLE_ID}'
      {_campaign_filter_core}
    GROUP BY 1
  )
SELECT
  COALESCE(m.date, l.date) AS date,
  COALESCE(m.attributed_installs, 0) AS attributed_installs,
  COALESCE(l.leaked_installs, 0) AS leaked_installs,
  ROUND(SAFE_DIVIDE(
    COALESCE(l.leaked_installs, 0),
    COALESCE(m.attributed_installs, 0) + COALESCE(l.leaked_installs, 0)
  ) * 100, 2) AS leakage_rate_pct
FROM moloco_attributed m
FULL OUTER JOIN leaked_daily l ON m.date = l.date
ORDER BY 1 DESC
"""

df_7a = run_query(q_7a, '7-A CT Install Leakage (1h window)')
if not df_7a.empty:
    total_leaked = df_7a['leaked_installs'].sum()
    total_attr = df_7a['attributed_installs'].sum()
    print(f'  → Total: {total_leaked:,.0f} leaked / {total_attr + total_leaked:,.0f} total ({total_leaked / (total_attr + total_leaked) * 100 if (total_attr + total_leaked) > 0 else 0:.2f}%)')
    print('  → Daily CT install leakage (unattributed installs within 1h of click):')
    for _, row in df_7a.iterrows():
        flag = ' ⚠️' if row['leakage_rate_pct'] and row['leakage_rate_pct'] > 10 else ''
        print(f'     {row["date"]}:  attributed={row["attributed_installs"]:,.0f}  leaked={row["leaked_installs"]:,.0f}  leakage={row["leakage_rate_pct"]:.1f}%{flag}')
df_7a

✅ 7-A CT Install Leakage (1h window): 15 rows
  → Total: 1,046 leaked / 9,093 total (11.50%)
  → Daily CT install leakage (unattributed installs within 1h of click):
     2026-02-27:  attributed=0  leaked=26  leakage=100.0% ⚠️
     2026-02-26:  attributed=654  leaked=74  leakage=10.2% ⚠️
     2026-02-25:  attributed=605  leaked=56  leakage=8.5%
     2026-02-24:  attributed=513  leaked=57  leakage=10.0%
     2026-02-23:  attributed=454  leaked=53  leakage=10.4% ⚠️
     2026-02-22:  attributed=533  leaked=77  leakage=12.6% ⚠️
     2026-02-21:  attributed=583  leaked=67  leakage=10.3% ⚠️
     2026-02-20:  attributed=467  leaked=69  leakage=12.9% ⚠️
     2026-02-19:  attributed=405  leaked=53  leakage=11.6% ⚠️
     2026-02-18:  attributed=632  leaked=81  leakage=11.4% ⚠️
     2026-02-17:  attributed=567  leaked=90  leakage=13.7% ⚠️
     2026-02-16:  attributed=625  leaked=82  leakage=11.6% ⚠️
     2026-02-15:  attributed=714  leaked=115  leakage=13.9% ⚠️
     2026-02-14:  attributed=686  l

,date,attributed_installs,leaked_installs,leakage_rate_pct
0,2026-02-27,0,26,100.00
1,2026-02-26,654,74,10.16
2,2026-02-25,605,56,8.47
3,2026-02-24,513,57,10.00
4,2026-02-23,454,53,10.45
5,2026-02-22,533,77,12.62
6,2026-02-21,583,67,10.31
7,2026-02-20,467,69,12.87
8,2026-02-19,405,53,11.57
9,2026-02-18,632,81,11.36


In [83]:
#@title 7-B. Rejected install rate (fact_dsp_core)

q_7b = f"""
SELECT
  date_utc,
  campaign_id,
  SUM(installs) AS total_installs,
  SUM(installs_rejected) AS total_rejected_installs,
  ROUND(SAFE_DIVIDE(SUM(installs_rejected), SUM(installs) + SUM(installs_rejected)) * 100, 2) AS rejection_rate_pct
FROM `moloco-ae-view.athena.fact_dsp_core`
WHERE date_utc BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 14 DAY) AND DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)
  AND advertiser.mmp_bundle_id = '{MMP_BUNDLE_ID}'
  {_campaign_filter_core}
GROUP BY 1, 2
ORDER BY 1 DESC
"""

df_7b = run_query(q_7b, '7-B Rejected Install Rate')
if not df_7b.empty:
    total_rejected = df_7b['total_rejected_installs'].sum()
    total_installs = df_7b['total_installs'].sum()
    avg_rate = df_7b['rejection_rate_pct'].fillna(0).mean()
    high_days = df_7b[df_7b['rejection_rate_pct'] > 10]
    if total_rejected > 0:
        print(f'  → Total rejected: {total_rejected:,.0f} / {total_installs + total_rejected:,.0f} ({avg_rate:.1f}% avg daily rejection)')
        if len(high_days) > 0:
            print(f'  → ⚠️ {len(high_days)} days with >10% rejection:')
            for _, row in high_days.iterrows():
                print(f'     {row["date_utc"]}: rejected={row["total_rejected_installs"]:,.0f}  rate={row["rejection_rate_pct"]:.1f}%')
    else:
        print(f'  → No rejected installs in the last 14 days ✅')
df_7b

✅ 7-B Rejected Install Rate: 14 rows
  → Total rejected: 427 / 8,474 (5.1% avg daily rejection)


,date_utc,campaign_id,total_installs,total_rejected_installs,rejection_rate_pct
0,2026-02-26,GK2fIdNthVl1rtyn,654,30,4.39
1,2026-02-25,GK2fIdNthVl1rtyn,605,30,4.72
2,2026-02-24,GK2fIdNthVl1rtyn,513,36,6.56
3,2026-02-23,GK2fIdNthVl1rtyn,454,24,5.02
4,2026-02-22,GK2fIdNthVl1rtyn,533,34,6.00
5,2026-02-21,GK2fIdNthVl1rtyn,583,34,5.51
6,2026-02-20,GK2fIdNthVl1rtyn,467,36,7.16
7,2026-02-19,GK2fIdNthVl1rtyn,405,16,3.80
8,2026-02-18,GK2fIdNthVl1rtyn,632,35,5.25
9,2026-02-17,GK2fIdNthVl1rtyn,567,30,5.03


---
## Summary

In [85]:
#@title Diagnostic Summary

print(f'==========================================')
print(f'  Campaign Diagnostic Summary')
print(f'  Input bundle   : {BUNDLE_ID}')
print(f'  tracking_bundle: {TRACKING_BUNDLE}')
print(f'  mmp_bundle_id  : {MMP_BUNDLE_ID}')
print(f'  app_store_bundle: {APP_STORE_BUNDLE}')
print(f'  Campaign: {CAMPAIGN_ID or "(all)"}')
print(f'==========================================')

checks = [
    ('1-A. PA Status (iOS)',     'skipped' if APP_OS != 'IOS' else (not df_1a.empty)),
    ('1-B. PA match_type',      not df_1b.empty and df_1b['attribution_method'].str.contains('probabilistic|modeled', case=False, na=False).any() if not df_1b.empty else None),
    ('2. VT Install',            not df_2.empty and df_2['is_view_through'].any() if not df_2.empty else None),
    ('3. Revenue Postback',      not df_3.empty),
    ('4. Creative Configured',   not df_4a.empty and (df_4a['is_archived'] == False).any() if not df_4a.empty else None),
    ('4. Creative Impressions',  not df_4b.empty),
    ('5. Kakao Bizboard',        not df_5.empty),
    ('6. Bid Filter (pricing)',  'skipped' if not CAMPAIGN_ID else (not df_6a.empty)),
    ('6. Bid Filter (trace)',    'skipped' if not CAMPAIGN_ID else (not df_6b.empty)),
    ('7-A. CT Leakage (1h)',     not df_7a.empty and df_7a['leakage_rate_pct'].fillna(0).mean() < 10 if not df_7a.empty else None),
    ('7-B. Rejected Install',    df_7b.empty or df_7b['rejection_rate_pct'].fillna(0).mean() < 10),
]

for name, result in checks:
    if result == 'skipped':
        icon = '⏭️'
        status = 'N/A (Android)' if 'iOS' in name or 'PA Status' in name else 'Skipped (no campaign_id)'
    elif result is None:
        icon = '❓'
        status = 'No data'
    elif bool(result):
        icon = '✅'
        status = 'OK'
    else:
        icon = '⚠️'
        status = 'Check needed'
    print(f'  {icon} {name}: {status}')

  Campaign Diagnostic Summary
  Input bundle   : com.fatmerge.global
  tracking_bundle: com.fatmerge.global
  mmp_bundle_id  : com.fatmerge.global
  app_store_bundle: com.fatmerge.global
  Campaign: GK2fIdNthVl1rtyn
  ⏭️ 1-A. PA Status (iOS): N/A (Android)
  ✅ 1-B. PA match_type: OK
  ⚠️ 2. VT Install: Check needed
  ✅ 3. Revenue Postback: OK
  ✅ 4. Creative Configured: OK
  ✅ 4. Creative Impressions: OK
  ⚠️ 5. Kakao Bizboard: Check needed
  ✅ 6. Bid Filter (pricing): OK
  ✅ 6. Bid Filter (trace): OK
  ⚠️ 7-A. CT Leakage (1h): Check needed
  ✅ 7-B. Rejected Install: OK
